In [15]:
def build_gridworld():
    """
    (0,0)을 왼쪽 아래로 놓고,
    rows=4, cols=4 크기의 격자를 예로 들어 설명합니다.
    
    - start_state = (0,0)
    - blocked = (1,1)
    - terminal_plus = (3,3)   # +1 보상
    - terminal_minus = (2,3)  # -1 보상
    
    U=(+1,0), D=(-1,0), L=(0,-1), R=(0,+1) 로 이동.
    """
    rows, cols = 3, 4

    # 관심 위치들
    start_state = (0, 0)
    blocked = (1, 1)
    terminal_plus = (2, 3)
    terminal_minus = (1, 3)

    # 가능한 행동
    actions = ['U', 'D', 'L', 'R']

    # (행 변화, 열 변화)
    moves = {
        'U': (1, 0),  # 위로
        'D': (-1, 0),  # 아래로
        'L': (0, -1),  # 왼쪽
        'R': (0, 1)  # 오른쪽
    }

    # 격자 범위 체크용
    def in_bounds(r, c):
        return 0 <= r < rows and 0 <= c < cols

    # 상태 집합(막힌 칸 제외)
    states = []
    for r in range(rows):
        for c in range(cols):
            if (r, c) == blocked:
                continue
            states.append((r, c))

    # 전이 모델 P[s][a] = [(prob, next_state, reward), ... ]
    P = {}

    for s in states:
        P[s] = {}

        # 터미널 상태 체크
        if s == terminal_plus:
            P[s]['terminal'] = True
            for a in actions:
                # 이미 종료된 상태이므로 자기 자신으로 이동, 보상 0
                P[s][a] = [(1.0, s, 0)]
            continue
        elif s == terminal_minus:
            P[s]['terminal'] = True
            for a in actions:
                P[s][a] = [(1.0, s, 0)]
            continue
        else:
            P[s]['terminal'] = False

        # 일반 상태의 경우
        for a in actions:
            dr, dc = moves[a]
            nr, nc = s[0] + dr, s[1] + dc

            # 범위 내인지 및 막힌 칸 아닌지 확인
            if in_bounds(nr, nc) and (nr, nc) != blocked:
                # 터미널 칸에 도달하는 경우 보상 부여
                if (nr, nc) == terminal_plus:
                    reward = 1.0
                elif (nr, nc) == terminal_minus:
                    reward = -1.0
                else:
                    reward = 0.0
                next_s = (nr, nc)
            else:
                # 이동 불가능하면 제자리 머무르기, 보상 0
                next_s = s
                reward = 0.0

            P[s][a] = [(1.0, next_s, reward)]

    return states, actions, P


In [25]:
for s in states:
    # 터미널 상태라면 업데이트 생략하고 그대로 둠
    if P[s].get("terminal", False):
        print(P[s])
        continue

{'terminal': True, 'U': [(1.0, (1, 3), 0)], 'D': [(1.0, (1, 3), 0)], 'L': [(1.0, (1, 3), 0)], 'R': [(1.0, (1, 3), 0)]}
{'terminal': True, 'U': [(1.0, (2, 3), 0)], 'D': [(1.0, (2, 3), 0)], 'L': [(1.0, (2, 3), 0)], 'R': [(1.0, (2, 3), 0)]}


In [1]:
def value_iteration(states, actions, P, gamma=0.9, theta=1e-6):
    """
    Value Iteration 알고리즘을 이용해 V(s)를 추정하고
    결정론적 정책 pi(s)를 반환한다.
    """
    # 1. V(s) 초기화
    V = {s: 0.0 for s in states}

    while True:
        delta = 0
        # 2. 모든 상태 s에 대해 갱신
        for s in states:
            # 터미널 상태라면 업데이트 생략하고 그대로 둠
            if P[s].get("terminal", False):
                continue

            v_old = V[s]

            # 가능한 모든 행동에 대한 Q(s,a)를 계산한 뒤 그 중 최댓값으로 갱신
            action_values = []
            for a in actions:
                q_value = 0.0
                for (prob, s_next, reward) in P[s][a]:
                    q_value += prob * (reward + gamma * V[s_next])
                action_values.append(q_value)

            V[s] = max(action_values) if action_values else v_old
            delta = max(delta, abs(v_old - V[s]))

        if delta < theta:
            break

    # 3. 결정론적 정책 도출
    policy = {}
    for s in states:
        if P[s].get("terminal", False):
            policy[s] = None  # 터미널 상태에는 정책 정의 X
            continue

        best_a, best_q = None, float('-inf')
        for a in actions:
            q_value = 0.0
            for (prob, s_next, reward) in P[s][a]:
                q_value += prob * (reward + gamma * V[s_next])
            if q_value > best_q:
                best_q = q_value
                best_a = a
        policy[s] = best_a

    return V, policy

In [20]:
if __name__ == "__main__":
    # Gridworld 환경 구성
    states, actions, P = build_gridworld()

    # Value Iteration 적용
    V, policy = value_iteration(states, actions, P, gamma=0.9, theta=1e-5)

    # 결과 출력
    print("== 최종 가치 함수(V) ==")
    for s in sorted(V):
        print(f"V({s}) = {V[s]:.4f}")

    print("\n== 도출된 정책(pi) ==")
    for s in sorted(policy):
        print(f"π({s}) = {policy[s]}")

== 최종 가치 함수(V) ==
V((0, 0)) = 0.6561
V((0, 1)) = 0.7290
V((0, 2)) = 0.8100
V((0, 3)) = 0.7290
V((1, 0)) = 0.7290
V((1, 2)) = 0.9000
V((1, 3)) = 0.0000
V((2, 0)) = 0.8100
V((2, 1)) = 0.9000
V((2, 2)) = 1.0000
V((2, 3)) = 0.0000

== 도출된 정책(pi) ==
π((0, 0)) = U
π((0, 1)) = R
π((0, 2)) = U
π((0, 3)) = L
π((1, 0)) = U
π((1, 2)) = U
π((1, 3)) = None
π((2, 0)) = R
π((2, 1)) = R
π((2, 2)) = R
π((2, 3)) = None
